# Laboratory of Computational Physics, Mod. B

* Marco Agnolon, 1205461
* Andrea Perin, 1206078
* Carlo Polato, 1205688
* Tommaso Tabarelli, 1205459

* ## TOC <a class="anchor" id="TOC"></a>
    * ## [Discussion](#disc)
        * [Description of the problem and objective](#description)
        * [Data](#data1)
        * [Problems with the data](#data_probs)
        * [Changing the data](#data2)
        * [Clustering and time series building](#clust)
        * [NN architectures and data formatting](#NN_arch)
        * [A gaussian trial](#gauss)
        * [Results](#results)
        * [Comments](#comments)
    * ## [Code and images](#code)
        * [Simulated data](#fake)
        * [Creating foreshocks](#itsnouse)
        * [DBSCAN example](#cigars)
        * [Importing true data](#truedata)
        * [Flattening data (Gall-Peters projection)](#fisherman)
        * [Fault line clustering](#kmeans)
        * [Finding maxima and splitting data](#peakutils)
        * [Defining regression NN](#regNN)
        * [Redefining regression NN with custom loss](#customregNN)
        * [Logistic classifier NN with custom loss](#logclass)
        * [MLP regressor](#mlp)
        * [Gaussian process](#gp)

## Discussion <a class="anchor" id="disc"></a>

### Description of the problem and objective <a class="anchor" id="description"></a>

The aim of the project is to study time series by way of machine learning (and, in particular, deep learning), or, at least, get to understand the predictive power of (simple) Neural Networks when applied to time series, in particular of earthquake data.

[Back to Table of Contents](#TOC)

### Data <a class="anchor" id="data1"></a>

The initial data we were dealing with was generated through a simulation, based on epidemic spreading mechanisms and empirical laws observed in nature. The main features of such model are, essentially, the following:
* strong (magnitude >4.5) earthquakes, also referred to as *mainshocks*, are preceded with a certain probability by smaller earthquakes of increasing magnitude, named *foreshocks*. It should be noted that the existence of foreshocks (and, by extent, of their use as indication of future mainshocks) is still not unanimously accepted, and foreshocks themselves are under study. It seems that foreshocks are not a feature of exceptionally strong earthquakes (which appear to be more sudden events), and that they are more common in certain geographical areas;
* strong earthquakes are also followed by a "seismic tail" of smaller events, called *aftershocks*, the intensities of which decrease over time, and the spatial distribution of which seems to follow a certain empirical distribution.

In practice, data was generated through a C++ algorithm which was given to us as a black box; we limited ourselves to understanding the algorithm's parameters, and adjusting them in order to suit our needs. As the algorithm was not designed as to produce foreshocks, we tried to do so in a rather naive way: instead of coding the (admittedly overwhelmingly complicated) microscopic mechanisms that give way to progressively stronger earthquakes, we tried to "reflect" the time of some aftershock events with respect to the mainshock, thus time-symmetrizing some events. This (questionable) procedure can be regarded as *data enhancing*: we effectively added information in our model, turning aftershocks into foreshocks. However, we soon came to realize that this approach was in fact doomed to fail: the process through which we reflected the earthquakes was entirely random, so that the "foreshocks" did not follow any precise law. There was effectively nothing to learn from these foreshocks, and indeed, one may argue that their inclusion was more detrimental than beneficial, as they were essentially an addition of noise before the mainshock.


[Back to Table of Contents](#TOC)

### Problems with the data   <a class="anchor" id="data_probs"></a>

The data that was generated in this way had the following problems:
1. as was just argued, they may not contain any real information;
2. even if one was to use them, they are not "efficient": there are plenty of events after mainshocks, but few before them. This is something that generated data has in common with real data, but it just makes real data more desirable;
3. mainshocks are way above the baseline in terms of magnitude, making them easily identifiable; real data, on the other hand, is less clearly cut out.


[Back to Table of Contents](#TOC)

### Changing the data   <a class="anchor" id="data2"></a>

On the basis of the aforementioned problems, we decided to use real data. In particular, we decided to use the data collected by the USGS in North America during the years ranging from 1991 to 2019. This choice was motivated by the quality of the data: they seem to be in good accordance with the expected distribution found in literature. Moreover, the area covered by the data was sufficiently vast (the entirety of USA), and it contains a very important fault line, where earthquakes more commonly have a relevant magnitude (>4.5). Data collected on other areas, on the other hand, are not as homogeneously distributed, to the point that they do not show the expected number of earthquakes as a function of their magnitude.

The data we used have information about latitude, longitude, depth, magnitude and time (expressed in seconds starting from the first earthquake in the series). We decided to preprocess the data in the following way:
* we discarded depth, which was deemed non informative when compared to the other spatial information. Moreover, depth measures have exceptionally large errors (sometimes close to 10% or more), to the point that some datapoints have negative depths;
* we switched from a polar coordinates system on a sphere to a cartesian system on a plane, in order to make spatial clustering easier. The method we employed is the Peters' transformation, the advantage of which is that it preserves the distances (and, as a consequence, the areas). Other alterantives (such as the Mercator projection) were discarded because they do not have this property;
* the spatial data was then converted to differential form: the $x$ and $y$ information were converted into distances; the times, instead, were converted into days, in order to make their absolute values smaller (as an aid to the NN).


[Back to Table of Contents](#TOC)

### Clustering and time series building <a class="anchor" id="clust"></a>

Once the format of the data was set, we went on to decide how to feed them to the NN. This choice is of paramount inmportance, as it implicitly bounds the architecture of the NN itself.

The first idea was to divide the dataset into a grid of both space and time fixed windows, using previous tiles to predict events in next ones. This idea, however, was discarded because of the tradeoff between the size of the time windows, the spatial grid size and the quantity of events in each *tile*:
* using small spatial grid would lead us to make a more accurate prediction in terms of space, but would also imply having less data in each *tile*, making the prediction very hard to perform;
* using greater time resolution would lead us to have more data in each *tile*, but the predicitons in terms of time would become meaningless (it makes no sense to predict an earthquake within a 10 days window).
For these reasons we preferred to adopt the cluster grouping.

The first step was to group the data on a geographical basis: the whole time series (comprehensive of all the earthquakes, regardless of their location) has too big of jumps between events for it to be feasibly learned by any sensible NN. It was then deemed essential to include some form of inductive bias in our problem: namely, the spatial relationships between earthquakes, which are linked by them belonging to the same fault line. This essentially amounts to a clustering procedure. The first clustering method we decided to employ was DBSCAN, a density based algorithm which previously showed nice properties: it need not to know the number of clusters in advance, nor is it sensitive to outliers. However, it needs to be fed a scale parameter and a minimum size. The performance of DBSCAN was not as good as expected: the problem lies in its dependance on the scale parameter: the data is not homogeneously distributed (not spatially nor temporally), so a density based approach can not reflect the local properties of the dataset: different areas have different tipical scales. `A more practical reason is that DBSCAN requires lots of RAM, as multiple kernel failures have taught us.`

For these reasons, we gave up on DBSCAN, and turned to a more naive approach. We used k-means algorithm with a number of clusters that was in slight excess of the one we expected from an eye based analysis. The resulting clusters were then grouped together on the basis of their belonging to some specific (and well documented) fault lines. This hybrid procedure, while it may seem a bit too rough and a bit arbitrary, is indeed just a way of introducing some inductive bias in our data. This makes the hypothesis class of any statistical learning algorithm much more scoped, improving performance and reducing the amount of data needed in order to learn.

Once the fault line clustering was done, we decided to consider each of the clusters as a separate entity. As a result, each cluster contains a time series. In order to retrieve the mainshocks of each cluster, we used a peak-finding algorithm, the likes of those employed in signal processing: given a set of "time windows", it retrieves the maximum inside the window, ignoring any other peak, and then slides the window forward: this kind of procedure was decided in order to avoid having too short time series (if 2 events above the threshold were near, they would generate 2 time series, the second being very short; as a consequence, this second time series would be not usable to feed the NN).

As a last passage, we decided to split each cluster-time series on the basis of these maxima, grouping all previous events down to the the previous maximum, and then split each of these groups once again in chunks of 50 samples, since we needed an homogenous format for our data. The choice of 50 as the size of our snippets is the result of a compromise between the meaningfullness of each snippet and their number: a small size would result in many snippets with low information quantity, while a larger size would end up in too few snippets, despite their informativeness. The result of this procedure was a set of snippets of 50 samples each, the majority of which ended with a small or otherwise unremarkable earthquake, while a fraction ended with what was regarded as a mainshock. Consequently, the series belong to roughly 2 main types:
* *background series*, which represent the majority of them, in which there are no significant events;
* *mainshock series*, in which the last event is the mainshock looked for, which we hope the algorithm can learn.


[Back to Table of Contents](#TOC)

### NN architectures and data formatting <a class="anchor" id="NN_arch"></a>

Three main approaches were attempted:

* ###### A predicting NN with an LSTM layer:
    The purpose of an LSTM layer is to correlate data on the basis of their time ordering. In our case, however, proper time ordering was discarded, in the sense that we did not take proper times into account, but just the order of the events (induced by the proper time ordering). The LSTM layer was followed by a simple set of dense layers and a 'relu' layer as output.
    The porpouse of this NN was that of trying to predict the magnitude of the immediately successive earthquake (namely, the 50-th one) basing on the previous 49.
        
* ###### A logistic classifier NN with an LSTM layer:
    This NN was much similar to the previous one: its architecture also involved a LSTM layer, but as output this NN has a sigmoid layer that gives a result that can be interpret as the probability of the label to be 1. Its porpouse was to give a probability estimation of the fact that the series in exam is leading to a big magnitude earthquake (as classified by the peak-findind algorithm).
    
* ###### An MLP regressor, using `sklearn` implementation:
    `sklearn` has an MLP regressor implementation, which we first used on the same snippets. Being a regressor, the MLP should be able to predict the next sample, given each of the 50-size snippets as a training point.

The label passed to the algorithms to make estimations/classifications were of 2 kinds:
* for regression (predicting NN and MLP regressor) we used for the series the magnitude of the last earthquake as label, regardless of it being "low magnitude" (in the case of *backgroud series*) or "high magnitude" (in the case the snippets ended with a mainshock);
* for the classification, we simply put label 0 to background snippets and 1 to snippets that contained mainshocks.

All the NN implemented by means of Keras included some `dropout` layers: these are special layers that implement the dropout procedure, a common way of avoiding overfitting in deep learning. It basically consists in 'powering off' random neurons at each training step. Dropout layers can also be used in order to output estimates of variance together with the estimate of the mean prediction.

Another important feature of our Keras models is the use of an early stopping criterion: this consists in using some control quantity as a metric of the generalisation properties of our algorithm. For instance, by taking the performance of the NN on a validation set (different from both the training set and the test set) as a control quantity, the early stopping procedure allows us to stop the training as soon as the loss on the validation set stops decreasing and plateaus. Indeed, evaluating performance only on training set leads to almost certain overfitting.

After having a look at the first results, we decided to try to implement a custom loss to see whether the NNs can weight false negatives more than false positives; this way we expected the NNs to predict better, but the results remained poor. In particular, while the average predicted magnitude increased appreciably, it was more of a baseline addition than a real improvement in predictive accuracy.

[Back to Table of Contents](#TOC)

### A gaussian trial <a class="anchor" id="gauss"></a>

The last attempt was using a gaussian process as a regressor. A gaussian process can be seen as a way of fitting a path integral to the data, and as such can be used as a regressor. The attempts we made were not successful, as predictions plummeted to 0 in a matter of a couple samples. However, the standard deviation of the predictions quickly increased, convincing us (once more) that no real predictions were made.

[Back to Table of Contents](#TOC)

### Results <a class="anchor" id="results"></a>

The results of all architectures were very poor. The explanations to this can be many:
* the data can be really non-informative; we are supposing that we are able to predict earthquakes basing on the past history in the considered zone, but they can be correlated in a very complex way that we are not able to catch with our simple models.
* our architectures are very simple and built with a small number of layers and neurons, which can be a major factor in poor predicting power. However, it should be noted that larger architecture wuold be more data hungry: the data at our disposal is too few to suffice those needs.

* as already mentioned, the analyzed time series can be grouped in roughly 2 main classes, *bkg series* and *mainshock series*; we think the algorithm fails at predicting also because there are too few *mainshock series* to learn compared with the large number of background series. The result of this unbalance is that the NNs tend to learn data distribution instead of data trend with respect to time, even if there is the LSTM layer. One may think that by pruning the dataset the algoritm can learn better; however, throwing data away should not be considered a good option because, as already mentioned, NNs are data hungry.


[Back to Table of Contents](#TOC)

### Comments <a class="anchor" id="comments"></a>

We decided not to use any kind of parallelization scheme because the ratio between costs in terms of time and effort and the increment in performance was disadvantageous. Indeed:
* using GPUs would need a more sophisticated way to write the code, in addition to installing all drivers and stuff needed to work with them;
* using `dask` to run parallel implementation was also considered, but the idea was scrapped because of the amount of data we were dealing with did not justify the additional heading that comes with a dask implementation; the resulting code might have been slower than the present one.

[Back to Table of Contents](#TOC)

## Code and Images <a class="anchor" id="code"></a>

### Simulated data <a class="anchor" id="fake"></a>


[Back to Table of Contents](#TOC)

### Creating foreshocks <a class="anchor" id="itsnouse"></a>


[Back to Table of Contents](#TOC)

### DBSCAN example <a class="anchor" id="cigars"></a>


[Back to Table of Contents](#TOC)

### Importing true data <a class="anchor" id="truedata"></a>


[Back to Table of Contents](#TOC)

### Flattening data (Gall-Peters projection) <a class="anchor" id="fisherman"></a>


[Back to Table of Contents](#TOC)

### Fault line clustering <a class="anchor" id="kmeans"></a>


[Back to Table of Contents](#TOC)

### Finding maxima and splitting data <a class="anchor" id="peakutils"></a>


[Back to Table of Contents](#TOC)

### Defining regression NN <a class="anchor" id="regNN"></a>


[Back to Table of Contents](#TOC)

### Redefining regression NN with custom loss <a class="anchor" id="customregNN"></a>


[Back to Table of Contents](#TOC)

### Logistic classifier NN with custom loss <a class="anchor" id="logclass"></a>


[Back to Table of Contents](#TOC)

### MLP regressor <a class="anchor" id="mlp"></a>


[Back to Table of Contents](#TOC)

### Gaussian process <a class="anchor" id="gp"></a>


[Back to Table of Contents](#TOC)

### 


[Back to Table of Contents](#TOC)